<a href="https://colab.research.google.com/github/haoruilee/js_competitions/blob/kaggle-upload-1/js-transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import polars as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
import os
import statistics as stat
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, mean_squared_error
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np

In [14]:
!ls ./drive/MyDrive/Timer_checkpoints

Timer_anomaly_detection_1.0.ckpt  Timer_forecast_1.0.ckpt  Timer_imputation_1.0.ckpt


In [2]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [3]:
!pwd

/content


In [3]:
import os

# Change directory to a specific path
os.chdir('/content/drive/MyDrive/checkpoints_kaggle_js')

# Verify the current working directory
print(os.getcwd())


/content/drive/MyDrive/checkpoints_kaggle_js


In [6]:
!git clone https://github.com/thuml/Large-Time-Series-Model.git


Cloning into 'Large-Time-Series-Model'...
remote: Enumerating objects: 292, done.
remote: Counting objects: 100% (208/208), done.
remote: Compressing objects: 100% (156/156), done.
remote: Total 292 (delta 105), reused 124 (delta 52), pack-reused 84 (from 1)
Receiving objects: 100% (292/292), 17.49 MiB | 24.37 MiB/s, done.
Resolving deltas: 100% (140/140), done.


In [10]:
!ls

jane-street-real-time-market-data-forecasting  len_5_d_model_64_nhead_32_num_layers_8_dropout_02
Large-Time-Series-Model			       Timer_forecast_1.0.ckpt


In [ ]:
# Import necessary libraries
import os
import time
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import polars as pl

# Add the Timer repository to the Python path
import sys
sys.path.append('./Large-Time-Series-Model')  # Adjust the path if needed

from models.Timer import Model  # Timer model

# Use CPU for debugging (switch to GPU after resolving issues)
device = torch.device("cpu")  # Use CPU for clear error messages
print(f"Using device: {device}")

# ==========================================
# 1. Load Data
# ==========================================
valid_from = 0

# Load data
start_time = time.time()
print("Loading data...")
jane_street_real_time_market_data_forecasting_path = "./jane-street-real-time-market-data-forecasting"  # Adjust path as needed
alltraindata = pl.scan_parquet(f"{jane_street_real_time_market_data_forecasting_path}/train.parquet")
train = alltraindata.filter(pl.col("date_id") >= valid_from).collect()
print(f"Data loaded in {time.time() - start_time:.2f} seconds")

# Prepare features
start_time = time.time()
print("Preparing features...")
feature_names = [f"feature_{i:02d}" for i in range(79)]
train_features = train.select(feature_names)
train_features = train_features.fill_null(strategy='forward').fill_null(0)
print(f"Features prepared in {time.time() - start_time:.2f} seconds")

# Prepare target data
start_time = time.time()
print("Preparing data...")
X = train_features.to_numpy()
y = train.select([col for col in train.columns if col.startswith('responder_')]).to_numpy()  # Use all responder columns
print(f"Data prepared in {time.time() - start_time:.2f} seconds")

# ==========================================
# 2. Define Dataset and DataLoader
# ==========================================
seq_len = 96  # Match the patch_len of the pre-trained model

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y, seq_len):
        self.X = X
        self.y = y
        self.seq_len = seq_len

    def __len__(self):
        return len(self.X) - self.seq_len

    def __getitem__(self, idx):
        X_seq = self.X[idx:idx + self.seq_len]
        y_seq = self.y[idx + self.seq_len - 1]  # 目标值是序列末尾的 responders
        return torch.tensor(X_seq, dtype=torch.float32), torch.tensor(y_seq, dtype=torch.float32)

# Define batch size
batch_size = 64

# Create train dataset and loader
train_dataset = TimeSeriesDataset(X, y, seq_len=seq_len)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

print(f"Train dataset size: {len(train_dataset)}")

# ==========================================
# 3. Define Timer Model with Configs
# ==========================================
class Configs:
    def __init__(self):
        self.task_name = "forecast"  # Task type
        self.ckpt_path = "./Timer_forecast_1.0.ckpt"  # Path to pre-trained checkpoint
        self.patch_len = 96  # Match the patch_len in the pre-trained model
        self.d_model = 1024  # Hidden dimension
        self.d_ff = 2048  # Feedforward network dimension
        self.e_layers = 8  # Number of Transformer encoder layers
        self.n_heads = 8  # Number of attention heads
        self.dropout = 0.1  # Dropout rate
        self.output_attention = False  # Output attention or not
        self.factor = 3  # Scaling factor for attention
        self.activation = 'gelu'  # Add activation function, e.g., 'gelu'

configs = Configs()

# Initialize the Timer model
model = Model(configs).to(device)
print("Timer model initialized.")

# ==========================================
# 4. Simplified Weighted Loss Function for Debugging
# ==========================================
def weighted_loss(outputs, targets):
    # print("Inside weighted_loss")  # 调试信息

    # 输出调试信息
    # print(f"Outputs shape: {outputs.shape}")  # 模型的预测输出
    # print(f"Targets shape: {targets.shape}")  # 目标值

    # 提取最后一个时间步的预测结果
    outputs = outputs[:, -1, :]  # 只取最后一个时间步
    # print(f"Outputs shape after slicing: {outputs.shape}")

    # 切片对齐
    if outputs.shape[-1] != targets.shape[-1]:
        outputs = outputs[:, :targets.shape[-1]]  # 对齐形状
        # print(f"Outputs shape after alignment: {outputs.shape}")

    # 计算损失
    loss = F.mse_loss(outputs, targets)

    # 为 `responder_6` 设置更高的权重
    if targets.shape[-1] > 6:
        weights = torch.ones_like(targets).to(device)
        weights[:, 6] = 20.0
        weighted_loss = loss * weights
    else:
        weighted_loss = loss

    return weighted_loss.mean()


# ==========================================
# 5. Debugging Training Loop
# ==========================================
# 训练循环
# 检查 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 初始化模型
model = Model(configs).to(device)
print("Timer model initialized.")

# 训练循环
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)

def save_checkpoint(epoch, model, optimizer, loss, checkpoint_dir):
    checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}.pth")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, checkpoint_path)
    print(f"Checkpoint saved at {checkpoint_path}")


epochs = 1000  # 训练 1000 个 epoch
for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for batch_idx, (batch_X, batch_y) in enumerate(train_loader):
        # 将数据转移到 GPU
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        # 输出调试信息
        print(f"Batch index: {batch_idx}")
        print(f"Batch_X shape: {batch_X.shape}")
        print(f"Batch_y shape: {batch_y.shape}")

        # 清零梯度
        optimizer.zero_grad()

        # 模型前向传播
        outputs = model(batch_X, None, None, None)
        print(f"Outputs shape: {outputs.shape}")

        # 检查输出和目标的数值范围
        print(f"Batch_y min: {batch_y.min().item()}, max: {batch_y.max().item()}")
        if torch.isnan(batch_y).any():
            print("Error: batch_y contains NaN values!")

        # 计算损失
        try:
            loss = weighted_loss(outputs, batch_y)
        except Exception as e:
            print(f"Error in weighted_loss: {e}")
            break

        # Debugging before backward
        print(f"Loss before backward: {loss.item()}")

        # 反向传播和优化
        try:
            loss.backward()
            optimizer.step()
        except Exception as e:
            print(f"Error during backward or step: {e}")
            break

        total_loss += loss.item()

        # Debugging: 打印前几个 batch 的信息
        if batch_idx >= 2:
            break

    if epoch % 100 == 0:
      # Save checkpoint at the end of each epoch
      save_checkpoint(epoch + 1, model, optimizer, total_loss / len(train_loader), './')
      print(f"SAVED!!! Epoch [{epoch + 1}/{epochs}], Average Loss: {total_loss / len(train_loader):.4f}")

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

    # 打印 GPU 使用情况
    if torch.cuda.is_available():
        print(f"GPU Memory Allocated: {torch.cuda.memory_allocated() / 1e6:.2f} MB")
        print(f"GPU Memory Cached: {torch.cuda.memory_reserved() / 1e6:.2f} MB")

torch.save(model.state_dict(), "./fine_tuned_timer_model.pth")
print("Fine-tuned Timer model saved successfully.")


Using device: cpu
Loading data...
Data loaded in 28.50 seconds
Preparing features...
Features prepared in 4.31 seconds
Preparing data...
Data prepared in 2.13 seconds
Train dataset size: 47127242
loading model:  ./Timer_forecast_1.0.ckpt


/content/drive/MyDrive/checkpoints_kaggle_js/./Large-Time-Series-Model/models/Timer.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sd = torch.load(self.ckpt_path, map_

Timer model initialized.
Using device: cuda
loading model:  ./Timer_forecast_1.0.ckpt
Timer model initialized.
Batch index: 0
Batch_X shape: torch.Size([64, 96, 79])
Batch_y shape: torch.Size([64, 9])
Outputs shape: torch.Size([64, 96, 79])
Batch_y min: -5.0, max: 5.0
Loss before backward: 5.7667107582092285
Batch index: 1
Batch_X shape: torch.Size([64, 96, 79])
Batch_y shape: torch.Size([64, 9])
Outputs shape: torch.Size([64, 96, 79])
Batch_y min: -2.7249138355255127, max: 2.9662516117095947
Loss before backward: 4.596888542175293
Batch index: 2
Batch_X shape: torch.Size([64, 96, 79])
Batch_y shape: torch.Size([64, 9])
Outputs shape: torch.Size([64, 96, 79])
Batch_y min: -5.0, max: 5.0
Loss before backward: 6.186418056488037
Checkpoint saved at ./checkpoint_epoch_1.pth
SAVED!!! Epoch [1/1000], Average Loss: 0.0000
Epoch 1/1000, Loss: 0.0000
GPU Memory Allocated: 1131.19 MB
GPU Memory Cached: 3323.99 MB
Batch index: 0
Batch_X shape: torch.Size([64, 96, 79])
Batch_y shape: torch.Size([6

In [19]:

# ==========================================
# 3. Define Timer Model with Configs
# ==========================================
class Configs:
    def __init__(self):
        self.task_name = "forecast"  # Task type
        self.ckpt_path = "./Timer_forecast_1.0.ckpt"  # Path to checkpoint
        self.patch_len = 96  # Patch length
        self.d_model = 1024  # Hidden dimension
        self.d_ff = 2048  # Feedforward network dimension
        self.e_layers = 8  # Number of Transformer encoder layers
        self.n_heads = 8  # Number of attention heads
        self.dropout = 0.1  # Dropout rate
        self.output_attention = False  # Output attention or not
        self.factor = 3  # Scaling factor for attention (set to 3 by default)
        self.activation = 'gelu'  # Add activation function, e.g., 'gelu'


configs = Configs()

# Initialize the Timer model
model = Model(configs).to(device)
print("Timer model initialized.")

# ==========================================
# 4. Define Weighted Loss Function
# ==========================================
def weighted_loss(outputs, targets):
    loss = F.mse_loss(outputs, targets, reduction='none')

    # Find the index of 'responder_6' dynamically
    responder_columns = [col for col in train.columns if col.startswith('responder_')]
    responder_6_index = responder_columns.index('responder_6')

    # Apply a higher weight for 'responder_6'
    weights = torch.ones_like(targets).to(device)
    weights[:, responder_6_index] = 20.0  # Assign higher weight to 'responder_6'

    weighted_loss = loss * weights
    return weighted_loss.mean()

# ==========================================
# 5. Fine-Tune the Model
# ==========================================
# Define optimizer and learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)

# Training loop
epochs = 10  # Number of epochs
for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_X, None, None, None)  # Timer model expects additional inputs, pass None for unused

        # Compute loss
        loss = weighted_loss(outputs, batch_y)

        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

# ==========================================
# 6. Save the Fine-Tuned Model
# ==========================================
torch.save(model.state_dict(), "./fine_tuned_timer_model.pth")
print("Fine-tuned Timer model saved successfully.")


loading model:  ./Timer_forecast_1.0.ckpt
Timer model initialized.


<ipython-input-19-21edc1359b27>:29: UserWarning: Using a target size (torch.Size([64, 9])) that is different to the input size (torch.Size([64, 96, 79])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(outputs, targets, reduction='none')


RuntimeError: The size of tensor a (79) must match the size of tensor b (9) at non-singleton dimension 2

features.csv	   lags.parquet    sample_submission.csv  train.parquet
kaggle_evaluation  responders.csv  test.parquet


In [ ]:
!ls ./drive/jane-street-real-time-market-data-forecasting

ls: cannot access './drive/jane-street-real-time-market-data-forecasting': No such file or directory


In [10]:

jane_street_real_time_market_data_forecasting_path = "./drive/MyDrive/jane-street-real-time-market-data-forecasting"  # Set your path here
checkpoint_dir = "./drive/MyDrive/checkpoints_kaggle_js/len_5_d_model_64_nhead_32_num_layers_8_dropout_02"


In [7]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import polars as pl
import os
import pandas as pd

valid_from = 0

# Load data
start_time = time.time()
print("Loading data...")
alltraindata = pl.scan_parquet(f"{jane_street_real_time_market_data_forecasting_path}/train.parquet")
train = alltraindata.filter(pl.col("date_id") >= valid_from).collect()
print(f"Data loaded in {time.time() - start_time:.2f} seconds")

# Prepare features
start_time = time.time()
print("Preparing features...")
feature_names = [f"feature_{i:02d}" for i in range(79)]
train_features = train.select(feature_names)
train_features = train_features.fill_null(strategy='forward').fill_null(0)
print(f"Features prepared in {time.time() - start_time:.2f} seconds")

# Prepare data
start_time = time.time()
print("Preparing data...")
X = train_features.to_numpy()
y = train.select([col for col in train.columns if col.startswith('responder_')]).to_numpy()  # Use all responder columns
print(f"Data prepared in {time.time() - start_time:.2f} seconds")

# Define sequence length
seq_len = 5


# Custom weighted loss function to prioritize responder_6
def weighted_loss(outputs, targets):
    loss = F.mse_loss(outputs, targets, reduction='none')

    # Find the index of 'responder_6' dynamically
    responder_columns = [col for col in train.columns if col.startswith('responder_')]
    responder_6_index = responder_columns.index('responder_6')

    # Apply a higher weight for the 'responder_6' target column
    weights = torch.ones_like(targets).to(device)  # Ensure weights are on GPU
    responder_6_weight = 20.0  # Assign higher weight to responder_6
    weights[:, responder_6_index] = responder_6_weight

    weighted_loss = loss * weights
    return weighted_loss.mean()


# Custom Dataset Class for Lazy Sequence Generation
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y, seq_len):
        self.X = X
        self.y = y
        self.seq_len = seq_len

    def __len__(self):
        return len(self.X) - self.seq_len

    def __getitem__(self, idx):
        # Get sequence of features
        X_seq = self.X[idx:idx + self.seq_len]
        # Get the target for the end of the sequence
        y_seq = self.y[idx + self.seq_len]
        # Convert to tensor
        X_tensor = torch.tensor(X_seq, dtype=torch.float32)
        y_tensor = torch.tensor(y_seq, dtype=torch.float32)
        return X_tensor, y_tensor

# Create dataset and data loader
print('Create dataset and data loader')
dataset = TimeSeriesDataset(X, y, seq_len=seq_len)
batch_size = 64
start_time = time.time()
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
print(f"Dataset and DataLoader created in {time.time() - start_time:.2f} seconds")

# Set up device for multi-GPU environment
print('Set up device for multi-GPU environment')
if torch.cuda.is_available():
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs!")
        device = torch.device("cuda")  # Use all available GPUs
    else:
        device = torch.device("cuda")
else:
    device = torch.device("cpu")
    print("Warning: CUDA is not available. Training will run on CPU.")

# Transformer Model Definition
class TransformerModel(nn.Module):
    def __init__(self, input_size, d_model=64, nhead=32, num_layers=8, dropout=0.2, output_size=6):
        super(TransformerModel, self).__init__()
        self.input_projection = nn.Linear(input_size, d_model)
        self.pos_encoder = nn.Sequential(
            nn.Dropout(dropout)
        )
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=d_model * 4,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers
        )
        self.output_layer = nn.Linear(d_model, output_size)  # Output matches number of responder columns

    def forward(self, x):
        x = self.input_projection(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = self.output_layer(x[:, -1, :])  # Use the output from the last time step
        return x.squeeze()

# Instantiate the model
input_size = X.shape[1]  # Number of features per time step
output_size = y.shape[1]  # Number of responder columns
print('Create model')
start_time = time.time()
model = TransformerModel(input_size=input_size, d_model=64, nhead=2, num_layers=2, dropout=0.2, output_size=output_size)
print(f"Model created in {time.time() - start_time:.2f} seconds")

print('Move model to GPU')
# Set up DataParallel to use multiple GPUs
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
model.to(device)

# Loss and optimizer
criterion = torch.nn.MSELoss()  # Base loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)




Loading data...
Data loaded in 11.90 seconds
Preparing features...
Features prepared in 4.34 seconds
Preparing data...
Data prepared in 2.50 seconds
Create dataset and data loader
Dataset and DataLoader created in 0.00 seconds
Set up device for multi-GPU environment
Create model
Model created in 0.02 seconds
Move model to GPU


In [11]:
import matplotlib.pyplot as plt

print('create_dir')
os.makedirs(checkpoint_dir, exist_ok=True)

# Optimizer and Scheduler
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

# Early Stopping Parameters
early_stopping_patience = 10
best_val_loss = float('inf')
patience_counter = 0

# Track Loss for Plotting
train_losses = []
val_losses = []

# Training loop with gradient clipping and validation
for epoch in range(epochs):
    epoch_start_time = time.time()
    print(f"Start Epoch {epoch+1}/{epochs}")

    # Training Phase
    model.train()
    running_loss = 0.0
    for batch_idx, (batch_X, batch_y) in enumerate(data_loader):
        batch_start_time = time.time()

        # Move batch data to the appropriate device
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_X)

        # Compute weighted loss
        loss = weighted_loss(outputs, batch_y)

        # Skip NaN losses
        if torch.isnan(loss):
            print(f"Skipping batch {batch_idx} due to NaN loss")
            continue

        # Backward pass and optimization
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 3000 == 0:
            print(f"Batch {batch_idx + 1} completed in {time.time() - batch_start_time:.2f} seconds, Loss: {loss.item():.4f}")

    train_loss = running_loss / len(data_loader)
    train_losses.append(train_loss)

    # Validation Phase
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_batch_X, val_batch_y in val_loader:  # Ensure val_loader is defined
            val_batch_X, val_batch_y = val_batch_X.to(device), val_batch_y.to(device)
            val_outputs = model(val_batch_X)
            val_loss += weighted_loss(val_outputs, val_batch_y).item()
    val_loss /= len(val_loader)
    val_losses.append(val_loss)

    print(f"Epoch [{epoch+1}/{epochs}] completed in {time.time() - epoch_start_time:.2f} seconds")
    print(f"Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

    # Update Scheduler
    scheduler.step(val_loss)

    # Early Stopping Check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # Save the best model
        torch.save(model.state_dict(), "best_transformer_model.pth")
        print("Best model saved as best_transformer_model.pth")
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered.")
            break

    # Save checkpoint after every epoch
    checkpoint_path = os.path.join(checkpoint_dir, f"transformer_checkpoint_epoch_{epoch+1}.pth")
    torch.save(model.state_dict(), checkpoint_path)
    print(f"Checkpoint saved as {checkpoint_path}")

# Save the final trained model
torch.save(model.state_dict(), "transformer_model.pth")
print("Model saved as transformer_model.pth")

# Plot Losses
plt.figure()
plt.plot(range(1, len(train_losses) + 1), train_losses, label="Training Loss")
plt.plot(range(1, len(val_losses) + 1), val_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()


create_dir


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


NameError: name 'epochs' is not defined

In [6]:
import matplotlib.pyplot as plt

# Load checkpoint for the model
checkpoint_path = "checkpoints/transformer_checkpoint_epoch_14.pth"
model.load_state_dict(torch.load(checkpoint_path))
model.to(device)

# Optimizer and Scheduler
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

# Early Stopping Parameters
early_stopping_patience = 10
best_val_loss = float('inf')
patience_counter = 0

# Track Loss for Plotting
train_losses = []
val_losses = []

# Training loop with gradient clipping and validation
for epoch in range(15, epochs):
    epoch_start_time = time.time()
    print(f"Start Epoch {epoch+1}/{epochs}")

    # Training Phase
    model.train()
    running_loss = 0.0
    for batch_idx, (batch_X, batch_y) in enumerate(data_loader):
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()

        outputs = model(batch_X)
        loss = weighted_loss(outputs, batch_y)

        # Skip NaN losses
        if torch.isnan(loss):
            print(f"Skipping batch {batch_idx} due to NaN loss")
            continue

        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 3000 == 0:
            print(f"Batch {batch_idx + 1} completed, Loss: {loss.item():.4f}")

    train_loss = running_loss / len(data_loader)
    train_losses.append(train_loss)

    # Validation Phase
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_batch_X, val_batch_y in val_loader:  # Assuming val_loader is defined
            val_batch_X, val_batch_y = val_batch_X.to(device), val_batch_y.to(device)
            val_outputs = model(val_batch_X)
            val_loss += weighted_loss(val_outputs, val_batch_y).item()
    val_loss /= len(val_loader)
    val_losses.append(val_loss)

    print(f"Epoch [{epoch+1}/{epochs}] completed in {time.time() - epoch_start_time:.2f} seconds")
    print(f"Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

    # Update Scheduler
    scheduler.step(val_loss)

    # Early Stopping Check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # Save the best model
        torch.save(model.state_dict(), "best_transformer_model.pth")
        print("Best model saved as best_transformer_model.pth")
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered.")
            break

    # Save checkpoint after each epoch
    torch.save(model.state_dict(), f"checkpoints/transformer_checkpoint_restart_epoch_{epoch+1}.pth")
    print(f"Checkpoint saved as checkpoints/transformer_checkpoint_restart_epoch_{epoch+1}.pth")

# Plot Losses
plt.figure()
plt.plot(range(1, len(train_losses) + 1), train_losses, label="Training Loss")
plt.plot(range(1, len(val_losses) + 1), val_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()


<ipython-input-6-aefc53e62121>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path))


FileNotFoundError: [Errno 2] No such file or directory: 'checkpoints/transformer_checkpoint_epoch_14.pth'

In [ ]:
if run_type == 'Transformer':
    # 模型参数
    input_size = 79
    d_model = 512
    nhead = 8
    num_layers = 3
    dropout = 0.1

    # 初始化模型
    model = TransformerModel(
        input_size=input_size,
        d_model=d_model,
        nhead=nhead,
        num_layers=num_layers,
        dropout=dropout
    ).to(device)

    print_model_size(model)

    # 优化器设置
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
    loss_function = nn.MSELoss(reduction='none')

    # 训练循环
    epochs = 20
    best = float('-inf')
    degraded = 0
    best_model = model

    for epoch in range(epochs):
        train_loss, train_mse, train_r2 = train_model(
            model, train_loader, optimizer, loss_function, device
        )

        scheduler.step()
        val_mse, val_r2 = evaluate_model(model, val_loader)

        print(f'epoch {epoch}:')
        print(f'train loss {train_loss:.4f}, train_r2 {train_r2:.4f}, '
              f'train_mse {train_mse:.4f}')
        print(f'val_mse {val_mse:.4f}, val_r2 {val_r2:.4f}')
        print(f'lr: {scheduler.get_last_lr()[0]:.6f}')

        if val_r2 > best:
            best = val_r2
            best_model = copy.deepcopy(model)
            torch.save(best_model.state_dict(), f'./model/js_{run_type}_unnorm.pth')
            degraded = 0
        else:
            degraded += 1

        if degraded > 10:
            print("Early stopping triggered")
            break

        torch.cuda.empty_cache()

    model = best_model
    test_mse, test_r2 = evaluate_model(model, test_loader)
    print(f'test R2 score is {test_r2}')


In [ ]:
lags_ : pl.DataFrame | None = None

def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make predictions using either LSTM or Transformer model"""
    global lags_
    if lags is not None:
        lags_ = lags

    # 初始化预测DataFrame
    predictions = test.select(
        'row_id',
        pl.lit(0.0).alias('responder_6'),
    )

    # 选择特征列
    feature_cols = [f"feature_{i:02d}" for i in range(79)]

    # 处理测试数据
    test_features = test.select(feature_cols)
    test_features = test_features.fill_null(strategy='forward').fill_null(0)

    # 转换为tensor
    X_test = test_features.to_numpy()
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)

    # 进行预测
    model.eval()
    with torch.no_grad():
        pred = model(X_test_tensor).cpu().numpy()

    # 更新预测结果
    predictions = predictions.with_columns(pl.Series('responder_6', pred.ravel()))

    # 验证输出格式
    assert isinstance(predictions, (pl.DataFrame, pd.DataFrame))
    assert list(predictions.columns) == ['row_id', 'responder_6']
    assert len(predictions) == len(test)
    print(predictions)
    return predictions

In [ ]:
import os

import pandas as pd
import polars as pl

import kaggle_evaluation.jane_street_inference_server


inference_server = kaggle_evaluation \
                        .jane_street_inference_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
            '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )
